In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os
import csv
import copy
import collections

In [2]:
def count_gender(ID_list, gender_list):

	gender_dictionary = dict(zip(ID_list, gender_list))
	dictionary_values = gender_dictionary.values()

	count = 0
	for index, value in enumerate(dictionary_values):
		if value == "F":
			count = count + 1


	print "Number of females: ", count;
	print "Number of males: ", len(dictionary_values) - count
	print "\n"



In [3]:
#read in the data frame
#df = pd.read_csv('big_file_excel.csv')
df = pd.read_csv('newdata.csv')

In [4]:
def update_Term(list):
    new_list = []
    for index, value in enumerate(list):
        if value[len(value)-1] == "F":
            new_value = value[:2] + "c_F"
        if value[len(value)-1] == "W":
            new_value = value[:2] + "a_W"
        if value[len(value)-1] == "S":
            new_value = value[:2] + "b_S"
        new_list.append(new_value)
    return new_list

In [5]:
df.rename(columns = {'\xef\xbb\xbfUIDHash':'ID', '(No column name)':'grade'}, inplace = True)
df.columns.values

array(['ID', 'Term', 'MajorCode', 'Classification', 'ResTuit', 'Gender',
       'AdmitClass', 'EthnicityCode', 'Athlete', 'HonorsFlg', 'USresident',
       'subject', 'course', 'grade'], dtype=object)

In [6]:
df['Term'] = update_Term(df['Term'].values.tolist())

In [7]:
#removes '0' before course code and white space after course and subject
df['course'] = df['course'].map(lambda x: x.lstrip('0').lstrip(' ').rstrip(' '))
df['subject'] = df['subject'].map(lambda x: x.rstrip(' '))
df = df[df['subject'].isin(["MATH"])]
#COMPTNG

In [8]:
#Lists of the math major and pre-math major codes
math_codes = ["072", "8D", "540", "545", "54B", "778", "G", "536", "394"]
pre_math_codes = ["6Q", "8E", "737", "6S", "6T", "6P", "6R"]
term_list = ['11c_F', '12a_W', '12b_S', '12c_F', '13a_W', '13b_S', '13c_F', '14a_W', '14b_S', '14c_F', '15a_W', '15b_S']

In [9]:
term_list = [['00c_F', '01a_W', '01b_S', '01c_F', '02a_W', '02b_S', '02c_F', '03a_W', '03b_S', '03c_F', '04a_W', '04b_S'],
             ['01c_F', '02a_W', '02b_S', '02c_F', '03a_W', '03b_S', '03c_F', '04a_W', '04b_S', '04c_F', '05a_W', '05b_S'],
             ['02c_F', '03a_W', '03b_S', '03c_F', '04a_W', '04b_S', '04c_F', '05a_W', '05b_S', '05c_F', '06a_W', '06b_S'],
             ['03c_F', '04a_W', '04b_S', '04c_F', '05a_W', '05b_S', '05c_F', '06a_W', '06b_S', '06c_F', '07a_W', '07b_S'],
             ['04c_F', '05a_W', '05b_S', '05c_F', '06a_W', '06b_S', '06c_F', '07a_W', '07b_S', '07c_F', '08a_W', '08b_S'],
             ['05c_F', '06a_W', '06b_S', '06c_F', '07a_W', '07b_S', '07c_F', '08a_W', '08b_S', '08c_F', '09a_W', '09b_S'],
             ['06c_F', '07a_W', '07b_S', '07c_F', '08a_W', '08b_S', '08c_F', '09a_W', '09b_S', '09c_F', '10a_W', '10b_S'],
             ['07c_F', '08a_W', '08b_S', '08c_F', '09a_W', '09b_S', '09c_F', '10a_W', '10b_S', '10c_F', '11a_W', '11b_S'],
             ['08c_F', '09a_W', '09b_S', '09c_F', '10a_W', '10b_S', '10c_F', '11a_W', '11b_S', '11c_F', '12a_W', '12b_S'],
             ['09c_F', '10a_W', '10b_S', '10c_F', '11a_W', '11b_S', '11c_F', '12a_W', '12b_S', '12c_F', '13a_W', '13b_S'],
             ['10c_F', '11a_W', '11b_S', '11c_F', '12a_W', '12b_S', '12c_F', '13a_W', '13b_S', '13c_F', '14a_W', '14b_S'],
             ['11c_F', '12a_W', '12b_S', '12c_F', '13a_W', '13b_S', '13c_F', '14a_W', '14b_S', '14c_F', '15a_W', '15b_S']]

In [10]:
all_math_codes = math_codes + pre_math_codes


In [11]:
full_term_list = ['00c_F', '01c_F', '02c_F', '03c_F', '04c_F', '05c_F', '06c_F', '07c_F', '08c_F', '09c_F', '10c_F', '11c_F']

student_profiles = []
for main_index, term in enumerate(full_term_list):
#Extract the subset of the data frame according to the term 11F (ostensibly freshmen)
    df_term = df[df['Term'] == term]

    #Extract from the same subset students who have admit class and classification  UFR
    df_term_UFR = df_term[df_term['AdmitClass'] == "UFR"]
    df_term_UFR = df_term_UFR[df_term_UFR['Classification'] == "UFR"]

    #Extract from the same subset students who have admit class and classification  UFR
    df_term_USO = df_term[df_term['AdmitClass'] == "USO"]
    df_term_USO = df_term_USO[df_term_USO['Classification'] == "USO"]

    #From the freshmen class beginning [the term], extract all math majors
    final_df_term_UFR = df_term_UFR[df_term_UFR['MajorCode'].isin(all_math_codes)]
    final_df_term_USO = df_term_USO[df_term_USO['MajorCode'].isin(all_math_codes)]

    final_df = pd.concat([final_df_term_UFR, final_df_term_USO])

    #Now df_term contains the students who started as fresmhmen in math in 2011. 
    #We will extract their ID values in order to follow them throughout their coursework
    set_of_IDs = list(set(final_df['ID'].values.tolist()))

    
    for index, ID_value in enumerate(set_of_IDs):
        specific_df = df[df['ID']==ID_value]

        specific_df.head()

        temp_term_list = specific_df['Term'].values.tolist()
        temp_course_list = specific_df['course'].values.tolist()
        temp_grade_list = specific_df['grade'].values.tolist()
        
        sum = 0
        for i, grade in enumerate(temp_grade_list):
            sum = sum + grade;
            
        gpa = float(sum)/len(temp_grade_list);
        
        temp_dict = {}
        temp_major = []
        for index, term in enumerate(term_list[main_index]):
            temp = []
            for index2, second_term in enumerate(temp_term_list):
                if term == second_term:
                    temp.append(temp_course_list[index2])
            temp_dict[term] = temp 
        temp_dict["major"] = specific_df['MajorCode'].values.tolist()[len(specific_df['MajorCode'].values.tolist())-1]
        temp_dict['Gender'] = specific_df['Gender'].values.tolist()[0]
        temp_dict['GPA'] = gpa
        student_profiles.append(temp_dict)

In [12]:
student_profiles

[{'00c_F': ['31A'],
  '01a_W': ['31B'],
  '01b_S': ['32A'],
  '01c_F': [],
  '02a_W': ['33A', '32B'],
  '02b_S': ['115A', '33B'],
  '02c_F': ['106', '110A'],
  '03a_W': ['167'],
  '03b_S': ['113', '132'],
  '03c_F': ['105A'],
  '04a_W': ['110B', '105B', '120A'],
  '04b_S': [],
  'GPA': 2.258823529411765,
  'Gender': 'M',
  'major': '540'},
 {'00c_F': ['31A'],
  '01a_W': ['31B'],
  '01b_S': ['32A'],
  '01c_F': [],
  '02a_W': [],
  '02b_S': [],
  '02c_F': [],
  '03a_W': [],
  '03b_S': [],
  '03c_F': [],
  '04a_W': [],
  '04b_S': [],
  'GPA': 1.5666666666666667,
  'Gender': 'M',
  'major': '540'},
 {'00c_F': ['31B'],
  '01a_W': [],
  '01b_S': [],
  '01c_F': [],
  '02a_W': [],
  '02b_S': [],
  '02c_F': [],
  '03a_W': [],
  '03b_S': [],
  '03c_F': [],
  '04a_W': [],
  '04b_S': [],
  'GPA': 3.0,
  'Gender': 'M',
  'major': '072'},
 {'00c_F': ['31B'],
  '01a_W': ['32A'],
  '01b_S': ['32B'],
  '01c_F': ['33A'],
  '02a_W': ['115A'],
  '02b_S': ['33B'],
  '02c_F': [],
  '03a_W': ['131A'],
  '03b

In [13]:
math_codes = ["072", "8D", "540", "545", "54B", "778", "G", "536", "394"]

applied_math_dictionaries = []
#Look at applied mathematics majors
for index, dictionary in enumerate(student_profiles):
    if dictionary["major"] == "540":
        applied_math_dictionaries.append(dictionary)

In [14]:
applied_math_dictionaries

[{'00c_F': ['31A'],
  '01a_W': ['31B'],
  '01b_S': ['32A'],
  '01c_F': [],
  '02a_W': ['33A', '32B'],
  '02b_S': ['115A', '33B'],
  '02c_F': ['106', '110A'],
  '03a_W': ['167'],
  '03b_S': ['113', '132'],
  '03c_F': ['105A'],
  '04a_W': ['110B', '105B', '120A'],
  '04b_S': [],
  'GPA': 2.258823529411765,
  'Gender': 'M',
  'major': '540'},
 {'00c_F': ['31A'],
  '01a_W': ['31B'],
  '01b_S': ['32A'],
  '01c_F': [],
  '02a_W': [],
  '02b_S': [],
  '02c_F': [],
  '03a_W': [],
  '03b_S': [],
  '03c_F': [],
  '04a_W': [],
  '04b_S': [],
  'GPA': 1.5666666666666667,
  'Gender': 'M',
  'major': '540'},
 {'00c_F': ['31B'],
  '01a_W': ['32A'],
  '01b_S': ['32B'],
  '01c_F': ['33A'],
  '02a_W': ['115A'],
  '02b_S': ['33B'],
  '02c_F': [],
  '03a_W': ['131A'],
  '03b_S': [],
  '03c_F': ['123', '110A'],
  '04a_W': ['120A', '131B'],
  '04b_S': [],
  'GPA': 2.8999999999999995,
  'Gender': 'M',
  'major': '540'},
 {'00c_F': ['35AH'],
  '01a_W': ['35BH'],
  '01b_S': ['32B', '33B'],
  '01c_F': ['113'],


In [15]:
len(student_profiles)

1353

In [16]:
len(applied_math_dictionaries)

415

In [17]:
sorted(applied_math_dictionaries[0].keys())

['00c_F',
 '01a_W',
 '01b_S',
 '01c_F',
 '02a_W',
 '02b_S',
 '02c_F',
 '03a_W',
 '03b_S',
 '03c_F',
 '04a_W',
 '04b_S',
 'GPA',
 'Gender',
 'major']

In [18]:
all_sequences = []
all_gpa = []
for main_index, dictionary in enumerate(applied_math_dictionaries):
    course_list = []
    for index, term in enumerate(sorted(applied_math_dictionaries[main_index].keys())):
        if term == "GPA":
            break;
        if len(applied_math_dictionaries[main_index][term]) > 1:
            course = 'and'.join(applied_math_dictionaries[main_index][term])
            course_list.append(course)
        if len(applied_math_dictionaries[main_index][term]) == 1:
            course = applied_math_dictionaries[main_index][term][0]
            course_list.append(course)
        for sec_index, course in enumerate(applied_math_dictionaries[main_index][term]):
            if course == "115A" or course == "115AH":
                break
        else:
            continue
        break
   
    string_of_courses = course_list[0]
    for index, course in enumerate(course_list):
        if index > 0:
            string_of_courses = string_of_courses + "-" + course
    string_of_courses = string_of_courses[:len(string_of_courses)]
    if("115A" in string_of_courses):
        all_sequences.append(string_of_courses)
        all_gpa.append(applied_math_dictionaries[main_index]['GPA'])

In [19]:
all_sequences

['31A-31B-32A-33Aand32B-115Aand33B',
 '31B-32A-32B-33A-115A',
 '31B-32A-32B-33A-33Band115AH',
 '31B-32A-32B-33A-33B-115AH',
 '31B-32A-32B-33A-33B-115A',
 '31B-32A-32B-33A-33B-115AHand136',
 '31B-31B-33A-32Band115A',
 '31A-31B-32A-32Band33A-115Aand33B',
 '31B-32A-32B-33A-33B-115A',
 '31A-31B-32A-32B-33A-115Aand33B',
 '31A-31B-32A-32B-33A-33B-115A',
 '31A-31B-32A-32B-33A-33B-61-123and115A',
 '35AH-35BH-33Band35CH-132and131AH-112  Mand131BHand135B-121and197-114Aand115AHand110AH',
 '32A-33A-32B-32B-115AHand33B',
 '1-31A-31B-32A-33A-115A',
 '31B-32A-33Aand32B-33B-115Aand131A',
 '31B-35BH-33Band35CH-33A-33A-115A',
 '31A-31B-32A-33Aand32B-32Band33B-115A',
 '31A-31B-32A-32B-33A-33B-115A',
 '35AH-32A-32Band33B-115A',
 '35AH-35BH-35CHand33B-131AHand115AH',
 '35AH-32A-32B-33B-115A',
 '31B-32A-32B-33A-33B-115A',
 '31B-32A-32B-33A-33B-33Band115A',
 '31B-32A-32B-33A-33B-61-115A',
 '31B-32A-32B-33A-33B-115A',
 '31B-32A-32B-33A-33B-61-115Aand131A',
 '31A-31B-32A-32B-61and33A-33B-170Aand131Aand115A',
 

In [20]:
all_gpa

[2.258823529411765,
 2.8999999999999995,
 2.9937500000000004,
 3.142857142857143,
 1.5333333333333334,
 3.6352941176470592,
 2.0714285714285716,
 3.407142857142857,
 3.8083333333333336,
 2.313333333333333,
 2.57,
 2.858823529411765,
 3.9586206896551723,
 2.1,
 2.34,
 3.7764705882352936,
 1.853333333333333,
 2.469230769230769,
 2.216666666666667,
 3.8066666666666666,
 3.176470588235294,
 3.3454545454545452,
 3.3,
 2.1615384615384614,
 2.3333333333333335,
 4.112499999999999,
 3.1999999999999997,
 2.314285714285714,
 1.8800000000000001,
 2.6187500000000004,
 3.191304347826087,
 3.0066666666666673,
 3.4899999999999998,
 1.1875,
 1.67,
 2.744444444444445,
 2.20625,
 4.05,
 3.3333333333333335,
 2.415,
 3.957142857142857,
 2.991666666666667,
 3.1888888888888887,
 3.6999999999999997,
 3.45,
 2.9437499999999996,
 2.857142857142857,
 3.3823529411764706,
 2.9250000000000003,
 3.3666666666666667,
 3.4071428571428575,
 3.189473684210526,
 3.9499999999999997,
 3.4785714285714286,
 3.2705882352941176

In [21]:
from collections import Counter
counter = Counter(all_sequences)

In [22]:
counter.keys()

['32A-33Aand32B-123and115AHand33B',
 '32A-33A-32B-33Band115AH',
 '32A-32BH-33AH-33B-115A',
 '31A-31B-32A-33Aand32B-32Band33B-115A',
 '33A-135-115A',
 '31B-32A-32B-33A-33B-115A',
 '32A-32B-33A-33B-115Aand170Aand142',
 '31A-31Band32A-32B-33Aand33B-134-115A',
 '35AH-32A-32Band33B-115A',
 '31B-32A-32B-33A-115Aand33B',
 '31B-32A-33Aand33B-32Band115A',
 '31A-31B-33Aand32A-32B-33B-115AHand180',
 '35AH-32A-32B-33B-115A',
 '32A-32B-33A-33B-113and115Aand61',
 '1-31A-31B-32A-33A-33Band32B-191and115A',
 '31A-31B-32A-32A-32B-33A-115Aand131Aand132',
 '32A-32B-33A-33Aand33B-33Band115A',
 '115A',
 '31B-32A-32B-33A-33Band115AH',
 '32A-32B-33A-106-197-115A',
 '32AH-32BH-33AH-33Band115AH',
 '32AH-32BH-33AH-33B-115A',
 '31B-32A-32Band33A-33Band115AHand199',
 '31A-31B-32A-33Aand32B-115Aand33B',
 '31A-31B-32A-32B-33A-33B-61-123and115A',
 '32A-32B-33A-33B-115Aand132',
 '31A-31B-31B-32B-33A-33B-115Aand106',
 '31B-32A-32B-33Band33A-115A',
 '31A-31B-32A-32B-33A-33Band115A',
 '33B-115A',
 '32A-32B-61and115AH',
 

In [23]:

for index, value in enumerate(counter.keys()):
    print (value), ",", counter[value]

32A-33Aand32B-123and115AHand33B , 1
32A-33A-32B-33Band115AH , 1
32A-32BH-33AH-33B-115A , 1
31A-31B-32A-33Aand32B-32Band33B-115A , 1
33A-135-115A , 1
31B-32A-32B-33A-33B-115A , 14
32A-32B-33A-33B-115Aand170Aand142 , 1
31A-31Band32A-32B-33Aand33B-134-115A , 1
35AH-32A-32Band33B-115A , 1
31B-32A-32B-33A-115Aand33B , 2
31B-32A-33Aand33B-32Band115A , 1
31A-31B-33Aand32A-32B-33B-115AHand180 , 1
35AH-32A-32B-33B-115A , 1
32A-32B-33A-33B-113and115Aand61 , 1
1-31A-31B-32A-33A-33Band32B-191and115A , 1
31A-31B-32A-32A-32B-33A-115Aand131Aand132 , 1
32A-32B-33A-33Aand33B-33Band115A , 1
115A , 1
31B-32A-32B-33A-33Band115AH , 1
32A-32B-33A-106-197-115A , 1
32AH-32BH-33AH-33Band115AH , 2
32AH-32BH-33AH-33B-115A , 4
31B-32A-32Band33A-33Band115AHand199 , 1
31A-31B-32A-33Aand32B-115Aand33B , 1
31A-31B-32A-32B-33A-33B-61-123and115A , 1
32A-32B-33A-33B-115Aand132 , 1
31A-31B-31B-32B-33A-33B-115Aand106 , 1
31B-32A-32B-33Band33A-115A , 2
31A-31B-32A-32B-33A-33Band115A , 1
33B-115A , 1
32A-32B-61and115AH , 1


In [24]:
count_list

NameError: name 'count_list' is not defined